In [295]:
import pandas as pd
import numpy as np

In [296]:
df = pd.read_csv('../data/raw/combined_data.csv')

In [297]:
df.head(10)

,Date,Time,Code,Value
0,05-20-1991,08:00,58,101
1,05-20-1991,08:00,33,5
2,05-20-1991,08:00,34,27
3,05-20-1991,12:00,60,89
4,05-20-1991,12:00,33,3
5,05-20-1991,18:00,62,94
6,05-20-1991,18:00,33,6
7,05-20-1991,22:00,48,109
8,05-21-1991,08:00,58,174
9,05-21-1991,08:00,33,5


In [298]:
df[["Value"]].isna().sum()

Value    33
dtype: int64

In [299]:
df.dropna(inplace=True)
df.isna().sum()

Date     0
Time     0
Code     0
Value    0
dtype: int64

In [300]:
print(df.columns.tolist())

['Date', 'Time', 'Code', 'Value']


In [301]:
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M', errors='coerce')

df = df[df['Time'].notna()]

df['Hour'] = df['Time'].dt.hour
df['Minute'] = df['Time'].dt.minute

# Cyclic encoding
df['Hour_sin'] = np.sin(2 * np.pi * df['Hour'] / 24)
df['Hour_cos'] = np.cos(2 * np.pi * df['Hour'] / 24)
df['Min_sin']  = np.sin(2 * np.pi * df['Minute'] / 60)
df['Min_cos']  = np.cos(2 * np.pi * df['Minute'] / 60)

print(df.head())

         Date                Time  Code Value  Hour  Minute      Hour_sin  \
0  05-20-1991 1900-01-01 08:00:00    58   101     8       0  8.660254e-01   
1  05-20-1991 1900-01-01 08:00:00    33     5     8       0  8.660254e-01   
2  05-20-1991 1900-01-01 08:00:00    34    27     8       0  8.660254e-01   
3  05-20-1991 1900-01-01 12:00:00    60    89    12       0  1.224647e-16   
4  05-20-1991 1900-01-01 12:00:00    33     3    12       0  1.224647e-16   

   Hour_cos  Min_sin  Min_cos  
0      -0.5      0.0      1.0  
1      -0.5      0.0      1.0  
2      -0.5      0.0      1.0  
3      -1.0      0.0      1.0  
4      -1.0      0.0      1.0  


In [302]:
df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
df.dropna(inplace=True)

In [303]:
X = df[["Code", "Hour_sin", "Hour_cos", "Min_sin", "Min_cos"]]
y = df[["Value"]]

In [304]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the resulting splits
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(23400, 5) (5851, 5) (23400, 1) (5851, 1)


In [305]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


/Users/shashankr/Development/Artificial Intelligence Projects/Zogle Labs/Learning Projects/Diabetes-prediction/learning-venv/lib/python3.12/site-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


In [306]:
from sklearn.metrics import mean_absolute_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'R² Score: {r2}')


Mean Absolute Error (MAE): 30.74129000623538
R² Score: 0.6771180076830843
